In [1]:
%pip install --upgrade polars

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl
from datetime import datetime

In [3]:
#not paarsing dates so I can first perform string operation on it
df_old = pl.read_csv("data/9june2023.csv", try_parse_dates=False)
df_new = pl.read_csv("data/2april2024.csv", try_parse_dates=False)

In [4]:
df_old = df_old.with_columns(sample_date = pl.lit("2023"))
df_new = df_new.with_columns(sample_date = pl.lit("2024"))

In [5]:
whole_df = df_old.vstack(df_new)

In [6]:
ex_rates = pl.read_csv("data/ex_rate.csv")

### 1. Removing duplicates

In [7]:
whole_df = whole_df.unique()

### 2. Cleaning comp_est

In [8]:
def split_ranges(df, col):
    '''
    calculates the length of a col_est, splits the longer str's from comp_est if they're an amalgamation of compensation ranges
    computes mean of the two numbers
    and saves it in a new column
    '''
    df = df.with_columns(len_comp = pl.col(col).str.len_chars())
    df = df.with_columns(est = pl.when((pl.col("len_comp")>10) & (pl.col(col).str.contains("[123456789]+[0]+[123456789]+[0]+")))
            .then(pl.col(col).str.extract_all("[123456789]+[0]+"))
            .otherwise(pl.concat_list(col, col))
    )
    df = df.with_columns(est_float = pl.col('est').cast(pl.List(pl.Float64)))
    df = df.with_columns(compensation = pl.col('est_float').list.mean())
    return df
    

In [9]:
def currency_exchange(df, rates_df):
    '''
    joins the main df with a currency exchange rates table
    calculates the compensation in dolars
    '''
    df = df.join(rates_df, how='left', on='country', coalesce=True)
    df = df.with_columns(comp_dol = pl.when(pl.col('rate').is_not_null())
                                        .then(pl.col('compensation')/pl.col('rate'))
                                        .otherwise(pl.col('compensation')))

    return df

In [10]:
def hourly_yearly(df, col, lower, upper):
    '''
    assumes that values between lower and upper
    are hourly wages and based on that calculates the yearly wage
    '''
    df = df.with_columns(pl.when((pl.col(col)>lower) & (pl.col(col)<upper))
                            .then(pl.col(col)*40*52)
                            .otherwise(pl.col(col))
    )
    return df

In [11]:
def monthly_yearly(df, col, lower, upper):
    '''
    assumes that values between lower and upper
    are hourly wages and based on that calculates the yearly wage
    '''
    df = df.with_columns(pl.when((pl.col(col)>lower) & (pl.col(col)<upper))
                            .then(pl.col(col)*12)
                            .otherwise(pl.col(col))
    )
    return df

In [12]:
def remove_comp_outliers(df, col, lower, upper):
    '''
    assumes extreme compensation numbers (0-9 and > 1_000_000) are a mistake
    and replaces them with a mean of compensation column
    '''
    mean_val = df.filter(pl.col(col)>10, pl.col(col)<upper).select(col).mean()
    df = df.with_columns(comp_dol = pl.when((pl.col(col)<lower) | (pl.col(col)>upper))
                                        .then(mean_val)
                                        .otherwise(pl.col(col))
    )
    return df

### 3. Missing categorical to "unknown"

In [13]:
def replace_missing(df, list_col, replace_with = "unknown"):
    '''
    replaces missing/null values with "unknown" category
    '''
    for col in list_col:
        df = df.with_columns(pl.col(col).fill_null(replace_with))

    return df

### 4. Cleaning job_names & language

In [14]:
def clean_jobnames(df):
    '''
    replaces "sr." and "jr." with respectively "senior" and "junior"
    '''
    df = df.with_columns(pl.col("job_name").str.to_lowercase().str.replace("sr.", "senior").str.replace("jr.", "junior"))

    return df

In [15]:
def clean_language(df):
    '''
    takes first two character from a column
    '''
    df = df.with_columns(pl.col('language').str.head(2))
    
    return df

### 5. Creating separate df for tags

In [16]:
def tags_df(df, list_cols):
    '''
    combines tags cols into one columns of strings
    '''
    for col in list_cols:
        df = df.with_columns(
            pl.col(col).str.extract_all(r"\w+").cast(pl.List(pl.String))
        )
    df = df.with_columns(all_tags=pl.col(list_cols[0])
                    .list.concat(list_cols[1:])
                    .list.unique()
                    .list.join(" "))

    return df
            
   

### 6. Adding a column of job_type (lead vs IC)

In [ ]:
def lead_ic(df):
    '''
    adds a column stating if a job is a leading position or IC
    '''
    df = df.with_columns(
        job_type=pl.when(pl.col("seniority").is_in(ic))
                .then(pl.lit("ic"))
                .when(pl.col("seniority").is_in(lead))
                .then(pl.lit("lead"))
                .otherwise(pl.lit("unclear"))
    )
    return df

### 7. Casting data types

In [17]:
selected_columns= ['job_name','hours','remote','company_name','education','seniority',
                'language','city','country','job_published_at','sample_date','comp_dol','all_tags']
cat_cols = ['hours','remote','company_name','education','seniority', 'language','city','country', 'sample_date', 'job_type']
str_col = ['job_name', 'all_tags']
data_col = ['job_published_at']
num_col = ['comp_dol']

In [18]:
def cast_dtypes(df, selected):
    '''
    changes the dtypes of specified columns
    '''
    clean_df = df.select(selected)
    for col in selected:
        if col in cat_cols:
            clean_df = clean_df.with_columns(pl.col(col).cast(pl.Categorical))
        if col in data_col:
            clean_df = clean_df.with_columns(pl.col(col).str.to_datetime().cast(pl.Date))
        if col in num_col:
            clean_df = clean_df.with_columns(pl.col(col).cast(pl.Int64))

    return clean_df

In [19]:
whole_df = split_ranges(whole_df, "comp_est")
whole_df = currency_exchange(whole_df, ex_rates)
whole_df = hourly_yearly(whole_df, 'comp_dol', 10, 200)
whole_df = monthly_yearly(whole_df, "comp_dol", 1_000, 10_000)
whole_df = remove_comp_outliers(whole_df, "comp_dol", 10, 1_000_000)
whole_df = replace_missing(whole_df, ['hours','remote','education','language'], replace_with="Unclear")
whole_df = replace_missing(whole_df , ['seniority'], replace_with="Unclear Seniority")
whole_df = clean_language(whole_df)
whole_df = clean_jobnames(whole_df)
whole_df = tags_df(whole_df,["tags_matched", "tag_categories", "categories"])
whole_df = lead_ic(whole_df)
whole_df = cast_dtypes(whole_df, selected_columns)

### 8. Setting data range/ Removing date outliers

In [20]:
clean_df = whole_df.filter(pl.col('job_published_at').is_between(datetime(2020, 12, 31), datetime(2024, 4, 2)))


### 9. Saving csv's 

In [21]:
clean_df = clean_df.rename({"company_name" : "company"})

['job_name',
 'hours',
 'remote',
 'company_name',
 'education',
 'seniority',
 'language',
 'city',
 'country',
 'job_published_at',
 'sample_date',
 'comp_dol',
 'all_tags']

In [22]:
clean_df.write_csv("data/analysis_df.csv", separator=",") #df for analysis

In [23]:
comp_df = clean_df.drop_nulls(subset = ["comp_dol", "all_tags"])

In [24]:
comp_df.write_csv("data/comp_df.csv", separator=",") #df for modelling